# Transform your samples posteriors to latex tables

Here....

@Author: P. Palma-Bifani

In [20]:
import glob
import numpy as np
import pandas as pd
import corner
import pickle

In [6]:
# test_file to check the functions and the workflow
path_test_file = '/Users/ppalmabifani/Desktop/exoAtm/c_ForMoSA/sub_modules/dF_to_latex/'

In [108]:


def formating_posteriors_values(latex_list_param, samples_directory, quantiles=[0.16, 0.5, 0.84], cifrassignificativas=[1,1,1], evidence=True):
    '''
    Desk
    
    Inputs:
    latex_list_param_name (str) :names of the parameters, just for control for now...
    samples_directory (str) :file name with path in string format
    quantiles                   (=[0.16, 0.5, 0.84])
    cifrassignificativas        (=[1,1,1])
    bayesian_evidence           (=True)

    Outputs:
    latex_params_list a list of strings that can be apppend for a latex table and the bayesian evidence params if wanted

    '''
    ## Open the sample
    with open(samples_directory, 'rb') as f1:
        result  = pickle.load(f1)
        samples = result.samples
        weights = result.weights
        
    ## Calculate quadriles and save param+deviation as string
    latex_params_list = []
    # once per variable
    for l in range(len(latex_list_param)):
        q16, q50, q84 = corner.quantile(samples[:,l], quantiles)
        qsub, qup     = np.abs(q50-q16), np.abs(q84-q50)

        latex_value_1 = round(q50, cifrassignificativas[l])
        latex_value_2 = round(qup, cifrassignificativas[l])
        latex_value_3 = round(qsub, cifrassignificativas[l])
        latex_value = '$' + str(latex_value_1) + '^{+' +  str(latex_value_2) + '}_{-' +str(latex_value_3) +'}$'
        latex_params_list.append(latex_value)
    
    if evidence == True:
            sample_logz = round(result['logz'],1)
            sample_logzerr = round(result['logzerr'],1)
            sample_h = round(result['h'],1)

            logz_value = '{logz} ± {logzerr}'.format(logz=sample_logz, logzerr=sample_logzerr)
            latex_params_list.append(logz_value)

            h_value    = '{h}'.format(h=sample_h )
            latex_params_list.append(h_value)

    return latex_params_list



def latex_pandas_table( latex_list_param, samples_directory,  evidencee=True):
    ''' Here
    '''

    columns_latex = ['Index']
    for i in range(len(latex_list_param)):
        columns_latex.append( latex_list_param[i])
    columns_latex.append('log(z)')
    columns_latex.append('h')
    
    latex_table = pd.DataFrame(columns = columns_latex)


    ## get strings\
    for m in range(len(samples_directory)):

        row_model = str(m)

        latex_posteriors_asym   = formating_posteriors_values(latex_list_param, samples_directory[m], cifrassignificativas=[1,2,2,2], evidence=evidencee)
        
        latex_row = [row_model] + latex_posteriors_asym
        latex_row_good = np.array(latex_row)\

        # Append row to the DataFrame using append()
        latex_table2 = latex_table.append(pd.Series(latex_row_good, index=latex_table.columns), ignore_index=True)\

    return latex_table2, columns_latex
        





In [109]:

## INPUTS & format definitions
testdata_samples_direct = np.array([path_test_file + 'testdata.pic'] )         # give list with all the .pic files of all wanted models
bayesian_evidence       = True

# For ForMoSA the list of parameters availables is
# 'T_{eff}','logg','[M/H]','C/O', 'R_{jup}', 'RV', 'A_v', 'vsin(i)', '\epsilon', 'BB_{rad}', 'BB_{Teff}'
latex_list_param_name   = ['T_{eff}','logg','[M/H]','C/O']           # must be in same order as the sample file


## get tables
# For this case only header and 1 row
latex_pd_dataframe, columns_latex   = latex_pandas_table(latex_list_param_name, testdata_samples_direct,evidencee=bayesian_evidence) 
# return pandas table to keep editong columns


# here to save in .txt directly to copy to latex
latex_pd_dataframe.to_csv(path_test_file+'testdata.txt', sep='&', line_terminator='\\\ \n', index=False, columns=columns_latex)


In [110]:
latex_pd_dataframe

,Index,T_{eff},logg,[M/H],C/O,log(z),h
0,0,$1551.8^{+228.5}_{-161.8}$,$3.24^{+0.67}_{-0.2}$,$0.49^{+0.27}_{-0.23}$,$0.72^{+0.07}_{-0.25}$,-1148.8 ± 0.4,17.2
